<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/Edge_list_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes

CPU times: user 252 ms, sys: 54.9 ms, total: 307 ms
Wall time: 25.6 s


In [2]:
# Load the Drive helper and mount
from google.colab import drive
%tensorflow_version 2.x
# This will prompt for authorization.
drive.mount('/content/drive')

TensorFlow 2.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%time
import datetime
import glob
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import distance
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import Point
import networkx as nx
from sys import getsizeof
from numba import jit
import dask.dataframe as dd
import dask.array as da
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar
%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 789 ms, sys: 82.7 ms, total: 871 ms
Wall time: 882 ms


In [4]:
client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/7905/1,Cluster Workers: 1 Cores: 4 Memory: 27.40 GB


In [5]:
path = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input'
path_output = '/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_output'

!ls "/content/drive/My Drive/05_Sync/FFE/FireNetwork/00_input"

buildings_raw.cpg      buildings_raw_pts.mshp  buildings_raw.shp
buildings_raw.dbf      buildings_raw_pts.prj   buildings_raw.shx
buildings_raw.prj      buildings_raw_pts.shp   GD_wind.csv
buildings_raw_pts.cpg  buildings_raw_pts.shx   source_target.csv
buildings_raw_pts.dbf  buildings_raw.qpj


In [0]:
def load_data(file_name, minx, miny, maxx, maxy):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(path, file_name), bbox=bbox)
    gdf_buildings['X'] = gdf_buildings.geometry.centroid.x
    gdf_buildings['Y'] = gdf_buildings.geometry.centroid.y

    # print(gdf_buildings.dtypes)
    max_extent = gdf_buildings.total_bounds
    data_size = getsizeof(gdf_buildings)/(1024.0**3)
    print("Shapefile extent : {}".format(max_extent))
    print("Asset loaded : {}".format(len(gdf_buildings)))
    print("Data size:{} GB'".format(data_size))
    # gdf_buildings.IgnProb_bl = 0.02
    # xmin,ymin,xmax,ymax = gdf_buildings.total_bounds

    # # Precision of float32 is sufficient for lat and lon
    # float_columns = ['SHAPE_Leng','SHAPE_Area',
    #                 'IgnProb_bl','RandProb']
    # gdf_buildings[float_columns] = gdf_buildings[float_columns].astype('float32')
    # int_columns = ['TARGET_FID','Combustibl',
    #                 'AU2013Num','RandProb']
    # gdf_buildings[int_columns] = gdf_buildings[int_columns].astype('int32')
    # data_size = getsizeof(gdf_buildings)/(1024.0**3)
    print("resized Data size:{} GB'".format(data_size))

    return gdf_buildings

In [7]:
%%time
# load data
gdf = load_data("buildings_raw.shp", 1748570, 5425500, 1749500, 5427600) # small
# gdf = load_data("buildings_raw.shp", 1745000, 5424148, 1750000, 5430000) # larger
# gdf = load_data("buildings_raw.shp", 1740508, 5420049, 1755776, 5443033) # whole
# gdf.plot(column='IgnProb_bl', cmap='hsv', legend=True)

Shapefile extent : [1748498.0152998  5425264.79535007 1749563.95249987 5427653.74945021]
Asset loaded : 1611
Data size:0.0002206386998295784 GB'
resized Data size:0.0002206386998295784 GB'
CPU times: user 395 ms, sys: 43.5 ms, total: 439 ms
Wall time: 450 ms


In [0]:
fid = gdf.TARGET_FID.values

t = [fid] * len(fid)
t = np.hstack(t)

In [9]:
len(t)

2595321

In [0]:
splits_of_dataframes = np.array_split(gdf, 100)
fid = gdf.TARGET_FID.values
x = gdf.X.values
y = gdf.Y.values

# @jit(parallel=True)
def source_target(splits_df, id_gdf, x_gdf, y_gdf):
  count = 0
  for df in splits_df[0]:
    count = count + 1
    print('split loaded : {} / {}'.format(count, len(splits_df)))

    # target
    t = [id_gdf] * len(id_gdf)
    t = np.hstack(t)
    print("item in target : {}".format(len(t)))
    x_target = [x_gdf] * len(id_gdf)
    x_target = np.hstack(x_target)
    y_target = [y_gdf] * len(id_gdf)
    y_target = np.hstack(y_target)

    # source
    s = np.repeat(id_gdf, len(id_gdf))
    x_source = np.repeat(x_gdf, len(id_gdf))
    y_source = np.repeat(y_gdf, len(id_gdf))
    print("item in source : {}".format(len(s)))

    # add to dataframe
    df = pd.DataFrame()
    df['source'] = s
    df['x_source'] = x_source
    df['y_source'] = y_source
    df['target'] = t 
    df['x_target'] = x_target
    df['y_target'] = y_target

    # filter on distance
    df['distance'] = ((df.x_source - df.x_target)**2 + (df.y_source - df.y_target)**2)**0.5
    df = df[(df.distance < 50) & (df.distance != 0)]
    df['azimuth'] = np.degrees(da.arctan2((df.x_source.values - df.x_target.values), (df.y_source.values - df.y_target.values)))
    df['bearing'] = (df.azimuth + 360) % 360

    df.to_csv(os.path.join(path_output, 'split_dataframes', 'split_{}'.format(count)))

  return

In [33]:
%%time
source_target(splits_of_dataframes, fid, x, y)

split loaded : 1 / 100
item in target : 2595321
item in source : 2595321
split loaded : 2 / 100
item in target : 2595321
item in source : 2595321
split loaded : 3 / 100
item in target : 2595321
item in source : 2595321
split loaded : 4 / 100
item in target : 2595321
item in source : 2595321
split loaded : 5 / 100
item in target : 2595321
item in source : 2595321
split loaded : 6 / 100
item in target : 2595321
item in source : 2595321
split loaded : 7 / 100
item in target : 2595321
item in source : 2595321
split loaded : 8 / 100
item in target : 2595321
item in source : 2595321
split loaded : 9 / 100
item in target : 2595321
item in source : 2595321
split loaded : 10 / 100
item in target : 2595321
item in source : 2595321
split loaded : 11 / 100
item in target : 2595321
item in source : 2595321
CPU times: user 6.57 s, sys: 467 ms, total: 7.04 s
Wall time: 7.06 s
